In [ ]:
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt
%cd yolov5

import torch
from IPython.display import Image, clear_output

clear_output()

In [ ]:
%cd /content
!curl -L "https://public.roboflow.com/ds/a8OpCzTP1X?key=D6cnshYPix" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

Streaming output truncated to the last 5000 lines.
 extracting: export/labels/1478900109306048491_jpg.rf.7f8266ddc28a351f0dcd20604c268259.txt  
 extracting: export/labels/1478900109306048491_jpg.rf.y4x0eVX2sO7ecR0TUaVr.txt  
 extracting: export/labels/1478900109878689818_jpg.rf.VDdJVseJIvOB61vpFirp.txt  
 extracting: export/labels/1478900109878689818_jpg.rf.bcebbb0d35bd86bda671f535b3f64031.txt  
 extracting: export/labels/1478900110449179205_jpg.rf.996b317597119cdcc563966e7c02b624.txt  
 extracting: export/labels/1478900110449179205_jpg.rf.BeHDkSoCfqcbL7l2e86o.txt  
 extracting: export/labels/1478900111020889294_jpg.rf.dac0676c053b218573c57367055fe8b2.txt  
 extracting: export/labels/1478900111020889294_jpg.rf.uUaiVa1Rn8K5eJTu2pvC.txt  
 extracting: export/labels/1478900111592085926_jpg.rf.58ddab1db87104355311ee1696d35449.txt  
 extracting: export/labels/1478900111592085926_jpg.rf.nA12WQhyH7O7E7ggQ4nn.txt  
 extracting: export/labels/1478900112162597702_jpg.rf.0fcefae4646f6879fce730488

In [ ]:
!mkdir train/
!mkdir train/images
!mkdir train/labels
!mkdir valid/
!mkdir valid/images
!mkdir valid/labels
!mkdir test/
!mkdir test/images
!mkdir test/labels

In [ ]:
import os
import shutil

# allImages = glob.glob("/export/images/*.jpg")
allImages = []
for filename in os.listdir("export/images/"):
    if filename.endswith(".jpg"):
      allImages.append(filename)
trainImages = []
valImages = []
trainLabels = []
valLabels = []
testImages = []
testLabels = []
print(len(allImages))
testCount = int(0.2 * len(allImages)/10)
trainCount = len(allImages)/10 - testCount-1
valCount = int(0.2 * trainCount)
trainCount = trainCount - valCount-1

print(trainCount, valCount, testCount)

# create training Dataset
j = 0
k = 0
for i in range(0,int(len(allImages)/10)):
  if i < trainCount: # create training images
    trainImages.append(allImages[i])
  elif i >= trainCount and i<(valCount + trainCount):
    valImages.append(allImages[i])
    # j += 1
  else:
    testImages.append(allImages[i])
    # k += 1

print(len(trainImages), len(valImages), len(testImages))

# moving training images
source = "export/images/"
destination = "train/images/"
for fileT in trainImages:
  shutil.move(source+fileT , destination+fileT)

source = "export/images/"
destination = "valid/images/"
for fileT in valImages:
  shutil.move(source+fileT , destination+fileT)

source = "export/images/"
destination = "test/images/"
for fileT in testImages:
  shutil.move(source+fileT , destination+fileT)

# moving labels
source = "export/labels/"
destination = "train/labels/"
for fileT in trainImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT , destination+fileT)

source = "export/labels/"
destination = "valid/labels/"
for fileT in valImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT , destination+fileT)

source = "export/labels/"
destination = "test/labels/"
for fileT in testImages:
  fileT = fileT.replace(".jpg",".txt")
  shutil.move(source+fileT , destination+fileT)

30000
1919.0 479 600
1919 479 602


In [ ]:
#extracting information from the roboflow file
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 11
names: ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']

In [ ]:
# define number of classes based on data.yaml
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
import cv2
import os
import numpy as np

def apply_clahe(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE to enhance contrast
    clahe = cv2.createCLAHE(clipLimit=6.0, tileGridSize=(8, 8))
    clahe_output = clahe.apply(gray)

    # Convert enhanced grayscale image back to BGR
    clahe_output_bgr = cv2.cvtColor(clahe_output, cv2.COLOR_GRAY2BGR)

    return clahe_output_bgr

# Function to perform contrast enhancement using CLAHE
def enhance_contrast(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE for contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)

    return enhanced

# Function to perform lane detection with ROI triangle for images
def detect_lanes_image(image):
    # Enhance contrast using CLAHE
    enhanced_image = enhance_contrast(image)

    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(enhanced_image, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blur, 50, 150)

    # Define ROI triangle and create mask
    height = image.shape[0]
    width = image.shape[1]
    mask = np.zeros_like(edges)
    triangle = np.array([[(150, height), (800, 450), (1800, height)]], np.int32)
    cv2.fillPoly(mask, triangle, 255)

    # Apply mask to edges image
    masked_edges = cv2.bitwise_and(edges, mask)

    # Detect lines using Hough Transform
    lines = cv2.HoughLinesP(masked_edges, 2, np.pi/180, 100, np.array([]), minLineLength=80, maxLineGap=6)

    # Overlay detected lines on original image
    result = np.copy(image)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(result, (x1, y1), (x2, y2), (0, 255, 255), thickness=5)

    return result
# Function to perform lane detection on video frames
def detect_lanes_video(frame):

    enhanced_frame = apply_clahe(frame)

    # Convert frame to HSV color space
    hsv = cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2HSV)

    # Define lower and upper bounds for yellow color in HSV
    low_yellow = np.array([18, 94, 140])
    up_yellow = np.array([48, 255, 255])

    # Define lower and upper bounds for white color in HSV
    low_white = np.array([0, 0, 200])
    up_white = np.array([180, 30, 255])

    # Create masks for yellow and white colors
    mask_yellow = cv2.inRange(hsv, low_yellow, up_yellow)
    mask_white = cv2.inRange(hsv, low_white, up_white)

    # Combine masks to detect both yellow and white lane markings
    combined_mask = cv2.bitwise_or(mask_yellow, mask_white)

    # Apply Canny edge detection to the combined mask
    edges = cv2.Canny(combined_mask, 50, 150)

    # Detect lines using Hough Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=60, maxLineGap=4)

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)

    return frame

# Path to the directory containing training images
train_image_dir = "train/images/"

# Iterate over each image in the directory for lane detection
for image_name in os.listdir(train_image_dir):
    image_path = os.path.join(train_image_dir, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to read image:", image_path)
        continue

    # Apply lane detection on image
    result_image = detect_lanes_image(image)

    # Save the result
    cv2.imwrite(image_path, result_image)

# Path to the directory containing testing images
test_image_dir = "test/images/"

# Iterate over each image in the directory
for image_name in os.listdir(test_image_dir):
    image_path = os.path.join(test_image_dir, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Unable to read image:", image_path)
        continue

    # Apply lane detection
    result = detect_lanes_image(image)

    # Save the result
    cv2.imwrite(image_path, result)

# Path to the input video
input_video_path = "project_video.mp4"

# Read the input video
video = cv2.VideoCapture(input_video_path)

# Get video frame width and height
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the output video writer for lane detection frames
lane_detection_output_filename = "lane_det_output.avi"
fourcc = cv2.VideoWriter_fourcc(*'XVID')
lane_detection_output = cv2.VideoWriter(lane_detection_output_filename, fourcc, 30.0, (frame_width, frame_height))

while True:
    ret, orig_frame = video.read()
    if not ret:
        break

    # Apply lane detection on original frame
    lane_detected_frame = detect_lanes_video(orig_frame)

    # Write frame with lane detection to output video
    lane_detection_output.write(lane_detected_frame)

# Release the VideoCapture and VideoWriter objects
video.release()
lane_detection_output.release()

# Read the input video again
video = cv2.VideoCapture(input_video_path)

# Define the output video writer for perspective-transformed frames
perspective_output_filename = "perspective_transformed_output.avi"
perspective_output = cv2.VideoWriter(perspective_output_filename, fourcc, 30.0, (frame_width, frame_height))

while True:
    ret, orig_frame = video.read()
    if not ret:
        break

    # Apply perspective transformation
    src_points = np.float32([[200, frame_height],
     [1080, frame_height],
      [540, 455],
       [720, 455]])
    dst_points = np.float32([
    [350, frame_height],  # Top-left corner of the trapezoid
    [1280 - 350, frame_height],  # Top-right corner of the trapezoid
    [350, 0],  # Bottom-left corner of the trapezoid
    [1280 - 350, 0]  # Bottom-right corner of the trapezoid
])
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    perspective_frame = cv2.warpPerspective(orig_frame, M, (frame_width, frame_height))

    # Apply lane detection on perspective-transformed frame
    lane_detected_perspective_frame = detect_lanes_video(perspective_frame)

    # Write perspective-transformed frame with lane detection to output video
    perspective_output.write(lane_detected_perspective_frame)

# Release the VideoCapture and VideoWriter objects
video.release()
perspective_output.release()
cv2.destroyAllWindows()

In [ ]:
# train yolov5s on custom data for 150 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 32 --epochs 150 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

/content/yolov5
2024-04-05 17:08:31.175556: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 17:08:31.175609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 17:08:31.264530: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=./models/custom_yolov5s.yaml, data=../data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi

In [ ]:
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 640 --conf 0.4 --source ../test/images

detect: weights=['runs/train/yolov5s_results/weights/best.pt'], source=../test/images, data=/data.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.5, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-295-gac6c4383 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7273488 parameters, 0 gradients
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/602 /content/test/images/1478019964687995430_jpg.rf.84582a0b8ca52e844106822430eb0104.jpg: 416x640 3 cars, 1 pedestrian, 80.6ms
image 2/602 /content/test/images/1478019977188852426_jpg.rf.2c692b8f11091ad527487338c581032e.jpg: 416x640 3 cars, 4 trafficLight-Reds, 1 trafficLight-RedLeft,

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'):
    display(Image(filename=imageName))
    print("\n")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --source ../lane_det_output.mp4 --conf 0.4

detect: weights=['runs/train/yolov5s_results/weights/best.pt'], source=../lane_det_output.mp4, data=/data.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.5, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-295-gac6c4383 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7273488 parameters, 0 gradients
video 1/1 (1/257) /content/lane_det_output.mp4: 384x640 (no detections), 78.2ms
video 1/1 (2/257) /content/lane_det_output.mp4: 384x640 (no detections), 9.3ms
video 1/1 (3/257) /content/lane_det_output.mp4: 384x640 (no detections), 8.8ms
video 1/1 (4/257) /content/lane_det_output.mp4: 384x640 (no detections), 9.9ms
WARNING ⚠️

In [ ]:
!pip install -q supervision ultralytics

import cv2
import numpy as np
import supervision as sv
from tqdm import tqdm
from ultralytics import YOLO
from supervision.assets import VideoAssets, download_assets
from collections import defaultdict, deque

download_assets(VideoAssets.VEHICLES)

SOURCE_VIDEO_PATH = "vehicles.mp4"
TARGET_VIDEO_PATH = "vehicles-result.mp4"
CONFIDENCE_THRESHOLD = 0.4
IOU_THRESHOLD = 0.5
MODEL_NAME = "runs/train/yolov5s_results/weights/best.pt"
MODEL_RESOLUTION = 640

SOURCE = np.array([
    [1252, 787],
    [2298, 803],
    [5039, 2159],
    [-550, 2159]
])

TARGET_WIDTH = 25
TARGET_HEIGHT = 250

TARGET = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

frame_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)
frame_iterator = iter(frame_generator)
frame = next(frame_iterator)

annotated_frame = frame.copy()
annotated_frame = sv.draw_polygon(scene=annotated_frame, polygon=SOURCE, color=sv.Color.red(), thickness=4)
sv.plot_image(annotated_frame)

class ViewTransformer:

    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points

        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

view_transformer = ViewTransformer(source=SOURCE, target=TARGET)

model = YOLO(MODEL_NAME)

video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)

# tracer initiation
byte_track = sv.ByteTrack(
    frame_rate=video_info.fps, track_thresh=CONFIDENCE_THRESHOLD
)

# annotators configuration
thickness = sv.calculate_dynamic_line_thickness(
    resolution_wh=video_info.resolution_wh
)
text_scale = sv.calculate_dynamic_text_scale(
    resolution_wh=video_info.resolution_wh
)
bounding_box_annotator = sv.BoundingBoxAnnotator(
    thickness=thickness
)
label_annotator = sv.LabelAnnotator(
    text_scale=text_scale,
    text_thickness=thickness,
    text_position=sv.Position.BOTTOM_CENTER
)
trace_annotator = sv.TraceAnnotator(
    thickness=thickness,
    trace_length=video_info.fps * 2,
    position=sv.Position.BOTTOM_CENTER
)

polygon_zone = sv.PolygonZone(
    polygon=SOURCE,
    frame_resolution_wh=video_info.resolution_wh
)

coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))

# open target video
with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:

    # loop over source video frame
    for frame in tqdm(frame_generator, total=video_info.total_frames):

        result = model(frame, imgsz=MODEL_RESOLUTION, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(result)

        # filter out detections by class and confidence
        detections = detections[detections.confidence > CONFIDENCE_THRESHOLD]
        detections = detections[detections.class_id != 0]

        # filter out detections outside the zone
        detections = detections[polygon_zone.trigger(detections)]

        # refine detections using non-max suppression
        detections = detections.with_nms(IOU_THRESHOLD)

        # pass detection through the tracker
        detections = byte_track.update_with_detections(detections=detections)

        points = detections.get_anchors_coordinates(
            anchor=sv.Position.BOTTOM_CENTER
        )

        # calculate the detections position inside the target RoI
        points = view_transformer.transform_points(points=points).astype(int)

        # store detections position
        for tracker_id, [_, y] in zip(detections.tracker_id, points):
            coordinates[tracker_id].append(y)

        # format labels
        labels = []

        for tracker_id in detections.tracker_id:
            labels.append(f"#{tracker_id}")

        # annotate frame
        annotated_frame = frame.copy()
        annotated_frame = trace_annotator.annotate(
            scene=annotated_frame, detections=detections
        )
        annotated_frame = bounding_box_annotator.annotate(
            scene=annotated_frame, detections=detections
        )
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame, detections=detections, labels=labels
        )

        # add frame to target video
        sink.write_frame(annotated_frame)
